In [ ]:
!pip install snownlp==0.11.1

In [ ]:
from snownlp import SnowNLP
from tqdm.notebook import tqdm
import pandas as pd

tqdm.pandas()

In [ ]:
df = pd.read_table("BosonNLP_sentiment_score.txt",sep= " ",names=['key','score'])

In [ ]:
key = df['key'].values.tolist()
score = df['score'].values.tolist()

In [ ]:
url_in = 'https://raw.githubusercontent.com/AlexRanzh/disaster-setiment-analysis/main/weibo-data-cleaned/mid_flood.csv'
url_post = 'https://raw.githubusercontent.com/AlexRanzh/disaster-setiment-analysis/main/weibo-data-cleaned/after_flood.csv'
url_pre = 'https://raw.githubusercontent.com/AlexRanzh/disaster-setiment-analysis/main/weibo-data-cleaned/pre_flood.csv'
df_in = pd.read_csv(url_in, usecols=['id', 'text_pre_list'])
df_post = pd.read_csv(url_post, usecols=['id', 'text_pre_list'])
df_pre = pd.read_csv(url_pre, usecols=['id', 'text_pre_list'])

Show some examples:

In [ ]:
print(df_in.head(5))
print(df_post.head(5))
print(df_pre.head(5))

                 id                                      text_pre_list
0  4689452229725981                       ['懂事', '孩子', '没有', '糖', '吃']
1  4689453065443305  ['老家', '房子', '听说', '都', '已经', '坍塌', '幸运的', '没有...
2  4689453438735200  ['真的', '救', '同学', '暴雨', '导致', '桥', '坍塌', '火车',...
3  4689453714769972  ['无人', '问晋', '河南', '暴雨', '晋城', '暴雨', '好多', '泥石...
4  4689454428588921  ['原本', '三个', '半个', '小时', '路程', '走', '小时', '下雨'...
                 id                                      text_pre_list
0  4690901877655172  ['微薄', '之力', '唯愿', '天佑', '中华', '日', '金榜题名', '定...
1  4690902155267478  ['包裹', '平遥', '古城', '不是', '艺术', '正在', '发生', '无法...
2  4690902101264925  ['我靠', '今天', '看到', '张艺兴', '飞天', '公益', '好多', '代...
3  4690902230502092  ['周深', '求生', '米', '一起', '捐助', '山西', '链接', '二维码...
4  4690902545859220  ['民族', '企业', '笔芯', '明目张胆', '拉', '踩', '偷偷', '斗鱼...
                 id                                      text_pre_list
0  4687749565644973  ['秋风', '猎', '红旗', '晨曦', '迎', '华诞', '庆祝', '新', ...
1  468

In [ ]:
def getscore(line):
    score_list  = [score[key.index(x)] for x in line if(x in key)]
    return sum(score_list)

In [ ]:
def helper(line):
    line = line.replace("'", '')
    line = line.replace("[", '')
    line = line.replace("]", '')
    out = line.split(", ")
    return out

In [ ]:
def normalize(score, m1, n1):
  if score > m1:
    score = m1
  if score < n1:
    score = n1

  if(score > 0):
    return score * 5 / m1
  elif(score < 0):
    return (score - n1) * 5 / (0 - n1) - 5
  else:
    return 0

In [ ]:
df_in["text_modified"] = df_in["text_pre_list"].apply(helper)
df_in["raw_score"] = df_in["text_modified"].progress_apply(getscore)

  0%|          | 0/15284 [00:00<?, ?it/s]

In [ ]:
df_in["normalized_score"] = df_in["raw_score"].progress_apply(normalize, m1 = 35, n1 = -11)

  0%|          | 0/15284 [00:00<?, ?it/s]

In [ ]:
df_post["text_modified"] = df_post["text_pre_list"].apply(helper)
df_post["raw_score"] = df_post["text_modified"].progress_apply(getscore)

  0%|          | 0/6701 [00:00<?, ?it/s]

In [ ]:
df_post["normalized_score"] = df_post["raw_score"].progress_apply(normalize, m1 = 70, n1 = -13)

  0%|          | 0/6701 [00:00<?, ?it/s]

In [ ]:
df_pre["text_modified"] = df_pre["text_pre_list"].apply(helper)
df_pre["raw_score"] = df_pre["text_modified"].progress_apply(getscore)

  0%|          | 0/5773 [00:00<?, ?it/s]

In [ ]:
df_pre["normalized_score"] = df_pre["raw_score"].progress_apply(normalize, m1 = 35, n1 = -13)

  0%|          | 0/5773 [00:00<?, ?it/s]

In [ ]:
df_pre.drop('text_modified', axis=1, inplace=True)

In [ ]:
print(df_pre)
print("Positive:", len([x for x in df_pre["normalized_score"] if x >= 0]))
print("Negative:", len([x for x in df_pre["normalized_score"] if x <= 0]))

                    id  ... normalized_score
0     4687749565644973  ...         5.000000
1     4687763977275545  ...         1.762369
2     4687775381586971  ...         4.184594
3     4687780363370863  ...         4.184594
4     4687781471715682  ...         0.871628
...                ...  ...              ...
5768  4689452192760337  ...         1.168998
5769  4689452004021748  ...         0.372305
5770  4689452183854318  ...         0.713905
5771  4689452096555216  ...        -0.098152
5772  4689452217401715  ...        -1.547848

[5773 rows x 4 columns]
Positive: 3660
Negative: 2258


In [ ]:
df_in.drop('text_modified', axis=1, inplace=True)

In [ ]:
print(df_in)
print("Positive:", len([x for x in df_in["normalized_score"] if x >= 0]))
print("Negative:", len([x for x in df_in["normalized_score"] if x <= 0]))

                     id  ... normalized_score
0      4689452229725981  ...         0.322505
1      4689453065443305  ...         0.081419
2      4689453438735200  ...        -2.852756
3      4689453714769972  ...        -3.084190
4      4689454428588921  ...        -2.558963
...                 ...  ...              ...
15279  4690901085193482  ...         0.429377
15280  4690901077328572  ...         4.920273
15281  4690901299628572  ...         1.013850
15282  4690901547092803  ...         1.787479
15283  4690901537916305  ...         0.248290

[15284 rows x 4 columns]
Positive: 11780
Negative: 3755


In [ ]:
df_post.drop('text_modified', axis=1, inplace=True)

In [ ]:
print(df_post)
print("Positive:", len([x for x in df_post["normalized_score"] if x >= 0]))
print("Negative:", len([x for x in df_post["normalized_score"] if x <= 0]))

                    id  ... normalized_score
0     4690901877655172  ...         0.519702
1     4690902155267478  ...         0.151657
2     4690902101264925  ...         3.181157
3     4690902230502092  ...         0.960492
4     4690902545859220  ...         0.016917
...                ...  ...              ...
6696  4696299851220769  ...         1.347835
6697  4696306218440728  ...         0.028235
6698  4696307552486120  ...         5.000000
6699  4696314686738706  ...         2.241076
6700  4696321896484116  ...         0.763268

[6701 rows x 4 columns]
Positive: 5946
Negative: 790


In [ ]:
df_pre.to_csv('pre_flood_with_score.csv',index=False)

In [ ]:
df_in.to_csv('mid_flood_with_score.csv',index=False)

In [ ]:
df_post.to_csv('post_flood_with_score.csv',index=False)